<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/CX_CB_ML_Implementation_(Random_Forest%2C_XGBoost%2C_LightGBM%2C_CatBoost%2C_LSTM%2C_GRU%2C_Prophet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Call Center ML Forecasting Pipeline - Phase 4
## Properly Cell-Structured Implementation

This notebook implements 7 ML models with proper cell structure for easy debugging and rerunning.
- All previous bugs fixed
- Checkpointing enabled
- GPU optimized
- 70+ features including holidays and market data


In [1]:

        !nvidia-smi

Fri Sep 19 21:27:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# Example: Move a tensor to the GPU
x = torch.randn(10, 10).to(device)

# Example: Move a model to the GPU
# model = YourModel().to(device)

Using GPU: Tesla T4


In [3]:

"""
Cell 1: Import all required libraries and configure GPU
"""

!pip install catboost optuna

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Core libraries
from datetime import datetime, timedelta
import json
import pickle
from typing import Dict, List, Tuple, Optional
import holidays

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# ML Libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Prophet
from prophet import Prophet

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
import optuna

# Configure GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU Available: {len(gpus)} GPU(s) detected")
    except RuntimeError as e:
        print(f"⚠️ GPU initialization error: {e}")
else:
    print("⚠️ No GPU detected, using CPU")

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)

print("✅ All libraries imported successfully!")


✅ GPU Available: 1 GPU(s) detected
✅ All libraries imported successfully!


In [4]:

"""
Cell 2: Define data loading function
"""
def load_call_center_data_v1_expanded(file_path='enhanced_eda_data.csv'):
    """
    Load call center data with market integration
    """
    print("📁 LOADING CALL CENTER DATA")
    print("=" * 45)

    try:
        # Load main data file
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        print(f"✅ Loaded {len(df)} records from {file_path}")

        # Auto-detect call volume column
        volume_cols = ['calls', 'Calls', 'call_volume', 'Call_Volume', 'volume', 'Volume']
        volume_col = None

        for col in volume_cols:
            if col in df.columns:
                volume_col = col
                break

        if volume_col is None:
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            volume_col = numeric_cols[0] if len(numeric_cols) > 0 else df.columns[0]

        print(f"🎯 Call volume column: {volume_col}")

        # Standardize column name
        if volume_col != 'calls':
            df = df.rename(columns={volume_col: 'calls'})

        # DATA CLEANING: Remove first and last rows
        print("🧹 DATA CLEANING: Removing first and last rows")
        original_len = len(df)
        if len(df) > 2:
            df = df.iloc[1:-1]
            print(f"   ✅ Cleaned: {original_len} → {len(df)} rows")

        # Market data integration
        expected_market_cols = [
            '^VIX_close', 'SPY_close', 'SPY_volume', 'QQQ_close', 'QQQ_volume',
            'DX-Y.NYB_close', 'GC=F_close', 'GC=F_volume', 'BTC-USD_close',
            'BTC-USD_volume', 'ETH-USD_close', 'ETH-USD_volume'
        ]

        existing_market_cols = [col for col in expected_market_cols if col in df.columns]

        if existing_market_cols:
            print(f"✅ Market data found: {len(existing_market_cols)} columns")

            # Enhanced market features
            if '^VIX_close' in df.columns:
                df['vix_high'] = (df['^VIX_close'] > df['^VIX_close'].quantile(0.8)).astype(int)
                df['vix_spike'] = (df['^VIX_close'].pct_change() > 0.2).astype(int)
                df['vix_returns'] = df['^VIX_close'].pct_change()
                df['vix_volatility'] = df['vix_returns'].rolling(7).std()

            if 'SPY_close' in df.columns:
                df['spy_returns'] = df['SPY_close'].pct_change()
                df['market_stress'] = (df['spy_returns'] < -0.02).astype(int)
                df['spy_volatility'] = df['spy_returns'].rolling(7).std()
                df['spy_momentum'] = df['SPY_close'].rolling(5).mean() / df['SPY_close'].rolling(20).mean()

            if 'BTC-USD_close' in df.columns:
                df['btc_returns'] = df['BTC-USD_close'].pct_change()
                df['crypto_volatility'] = df['btc_returns'].rolling(7).std()
                df['btc_extreme_move'] = (abs(df['btc_returns']) > 0.1).astype(int)

        print(f"\n📊 FINAL DATASET OVERVIEW")
        print(f"   Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
        print(f"   Total days: {len(df)}")
        print(f"   Call volume range: {df['calls'].min():.0f} to {df['calls'].max():.0f}")

        return df

    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None


In [5]:

"""
Cell 3: Load the actual data
"""
df = load_call_center_data_v1_expanded('enhanced_eda_data.csv')

if df is None:
    print("⚠️ ERROR: Could not load data!")
    print("Please ensure 'enhanced_eda_data.csv' exists in the current directory.")
else:
    print(f"✅ Data loaded successfully! Shape: {df.shape}")


📁 LOADING CALL CENTER DATA
✅ Loaded 978 records from enhanced_eda_data.csv
🎯 Call volume column: calls
🧹 DATA CLEANING: Removing first and last rows
   ✅ Cleaned: 978 → 976 rows
✅ Market data found: 12 columns

📊 FINAL DATASET OVERVIEW
   Date range: 2023-01-02 to 2025-09-03
   Total days: 976
   Call volume range: 3462 to 24724
✅ Data loaded successfully! Shape: (976, 30)


In [6]:

"""
Cell 4: Define feature engineering function
"""
def create_features(df: pd.DataFrame, target_col: str = 'calls',
                    country: str = 'US', state: str = None) -> pd.DataFrame:
    """
    Create comprehensive features for ML models
    """
    df = df.copy()

    # Handle Date index/column ambiguity - FIXED VERSION
    if df.index.name == 'Date' and 'Date' in df.columns:
        df = df.drop(columns=['Date']).reset_index()
    elif df.index.name == 'Date':
        df = df.reset_index()
    elif 'Date' not in df.columns:
        raise ValueError("No Date column or index found")

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # Basic temporal features
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['weekofyear'] = df['Date'].dt.isocalendar().week
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_month_start'] = df['Date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['Date'].dt.is_month_end.astype(int)

    # Cyclical encoding
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

    # Holiday features
    print("🗓️ Adding holiday features...")
    years = df['Date'].dt.year.unique()

    if country == 'US':
        holiday_list = holidays.US(state=state, years=years.tolist()) if state else holidays.US(years=years.tolist())
    else:
        holiday_list = holidays.CountryHoliday(country, years=years.tolist())

    df['is_holiday'] = df['Date'].dt.date.isin(holiday_list.keys()).astype(int)

    holiday_dates = pd.to_datetime(list(holiday_list.keys()))

    def days_to_nearest_holiday(date):
        if len(holiday_dates) == 0:
            return 999
        return min(abs((date - holiday_dates).days).min(), 30)

    df['days_to_nearest_holiday'] = df['Date'].apply(days_to_nearest_holiday)

    print(f"   ✅ Added {df['is_holiday'].sum()} holidays")

    # Lag features
    lag_features = [1, 2, 3, 7, 14, 21, 28, 30, 60, 90]
    for lag in lag_features:
        df[f'lag_{lag}'] = df[target_col].shift(lag)

    # Rolling window statistics
    rolling_windows = [7, 14, 21, 28, 60, 90]
    for window in rolling_windows:
        df[f'rolling_mean_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).mean()
        df[f'rolling_std_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).std()
        df[f'rolling_min_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).min()
        df[f'rolling_max_{window}'] = df[target_col].shift(1).rolling(window=window, min_periods=1).max()

    # Expanding features
    df['expanding_mean'] = df[target_col].shift(1).expanding(min_periods=1).mean()
    df['expanding_std'] = df[target_col].shift(1).expanding(min_periods=1).std()

    # Difference features
    df['diff_1'] = df[target_col].diff(1)
    df['diff_7'] = df[target_col].diff(7)
    df['diff_30'] = df[target_col].diff(30)

    # Trend features
    df['days_since_start'] = (df['Date'] - df['Date'].min()).dt.days

    # Interaction features
    df['dayofweek_month'] = df['dayofweek'] * df['month']
    df['is_weekend_month'] = df['is_weekend'] * df['month']

    return df


In [7]:

"""
Cell 5: Create features
"""
df_features = create_features(df, target_col='calls', country='US', state=None)
print(f"✅ Features created. Shape: {df_features.shape}")
print(f"Total features: {len([col for col in df_features.columns if col not in ['Date', 'calls']])}")


🗓️ Adding holiday features...
   ✅ Added 30 holidays
✅ Features created. Shape: (976, 90)
Total features: 88


In [8]:

"""
Cell 6: Define train/test split function
"""
def prepare_train_test_split(df: pd.DataFrame,
                             test_size: int = 180,
                             target_col: str = 'calls') -> Tuple:
    """
    Prepare train/test split matching previous time series work
    """
    # Remove NaN values
    df_clean = df.dropna()

    # Split data
    train_size = len(df_clean) - test_size

    train_data = df_clean.iloc[:train_size].copy()
    test_data = df_clean.iloc[train_size:].copy()

    # Prepare features and targets
    feature_cols = [col for col in df_clean.columns if col not in ['Date', target_col]]

    X_train = train_data[feature_cols]
    y_train = train_data[target_col]
    X_test = test_data[feature_cols]
    y_test = test_data[target_col]

    # Store dates for plotting
    train_dates = train_data['Date']
    test_dates = test_data['Date']

    print(f"Train shape: X={X_train.shape}, y={y_train.shape}")
    print(f"Test shape: X={X_test.shape}, y={y_test.shape}")
    print(f"Train period: {train_dates.min()} to {train_dates.max()}")
    print(f"Test period: {test_dates.min()} to {test_dates.max()}")

    return X_train, X_test, y_train, y_test, train_dates, test_dates


In [9]:

"""
Cell 7: Create train/test split
"""
X_train, X_test, y_train, y_test, train_dates, test_dates = prepare_train_test_split(df_features, test_size=180)


Train shape: X=(706, 88), y=(706,)
Test shape: X=(180, 88), y=(180,)
Train period: 2023-04-02 00:00:00 to 2025-03-07 00:00:00
Test period: 2025-03-08 00:00:00 to 2025-09-03 00:00:00


In [10]:

"""
Cell 8: Define the main ML forecasting pipeline class
"""
class MLForecastingPipeline:
    """Comprehensive ML forecasting pipeline"""

    def __init__(self, X_train, X_test, y_train, y_test, train_dates=None, test_dates=None):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.train_dates = train_dates  # Fixed: Added dates
        self.test_dates = test_dates    # Fixed: Added dates
        self.models = {}
        self.predictions = {}
        self.metrics = {}
        self.best_params = {}

        # Scalers for neural networks
        self.scaler_X = StandardScaler()
        self.scaler_y = MinMaxScaler()

    def calculate_metrics(self, y_true, y_pred, model_name):
        """Calculate comprehensive metrics"""
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = mean_absolute_percentage_error(y_true, y_pred) * 100

        return {
            'Model': model_name,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'R2': 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - y_true.mean()) ** 2))
        }


In [11]:

"""
Cell 9: Add Random Forest training method to pipeline
"""
def train_random_forest(self, tune_hyperparams=True):
    """Train Random Forest with optional hyperparameter tuning"""
    print("\n" + "="*50)
    print("Training Random Forest...")

    if tune_hyperparams:
        param_dist = {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }

        rf = RandomForestRegressor(random_state=42, n_jobs=-1)
        tscv = TimeSeriesSplit(n_splits=3)

        rf_search = RandomizedSearchCV(
            rf, param_dist, n_iter=10, cv=tscv,
            scoring='neg_mean_absolute_error',
            random_state=42, n_jobs=-1
        )
        rf_search.fit(self.X_train, self.y_train)

        self.models['RandomForest'] = rf_search.best_estimator_
        self.best_params['RandomForest'] = rf_search.best_params_
        print(f"Best params: {rf_search.best_params_}")
    else:
        self.models['RandomForest'] = RandomForestRegressor(
            n_estimators=200, max_depth=20, random_state=42, n_jobs=-1
        )
        self.models['RandomForest'].fit(self.X_train, self.y_train)

    self.predictions['RandomForest'] = self.models['RandomForest'].predict(self.X_test)
    self.metrics['RandomForest'] = self.calculate_metrics(
        self.y_test, self.predictions['RandomForest'], 'RandomForest'
    )
    print(f"RandomForest MAE: {self.metrics['RandomForest']['MAE']:.2f}")

# Attach method to class
MLForecastingPipeline.train_random_forest = train_random_forest


In [12]:

"""
Cell 10: Add XGBoost training method
"""
def train_xgboost(self, tune_hyperparams=True):
    """Train XGBoost with optional hyperparameter tuning"""
    print("\n" + "="*50)
    print("Training XGBoost...")

    if tune_hyperparams:
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            }

            model = xgb.XGBRegressor(**params, random_state=42)

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []
            for train_idx, val_idx in tscv.split(self.X_train):
                X_fold_train = self.X_train.iloc[train_idx]
                y_fold_train = self.y_train.iloc[train_idx]
                X_fold_val = self.X_train.iloc[val_idx]
                y_fold_val = self.y_train.iloc[val_idx]

                model.fit(X_fold_train, y_fold_train, eval_set=[(X_fold_val, y_fold_val)],
                          verbose=False)
                pred = model.predict(X_fold_val)
                scores.append(mean_absolute_error(y_fold_val, pred))

            return np.mean(scores)

        study = optuna.create_study(direction='minimize')
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study.optimize(objective, n_trials=10, show_progress_bar=False)

        self.best_params['XGBoost'] = study.best_params
        self.models['XGBoost'] = xgb.XGBRegressor(**study.best_params, random_state=42)
        self.models['XGBoost'].fit(self.X_train, self.y_train)
        print(f"Best params: {study.best_params}")
    else:
        self.models['XGBoost'] = xgb.XGBRegressor(
            n_estimators=200, max_depth=6, learning_rate=0.1, random_state=42
        )
        self.models['XGBoost'].fit(self.X_train, self.y_train)

    self.predictions['XGBoost'] = self.models['XGBoost'].predict(self.X_test)
    self.metrics['XGBoost'] = self.calculate_metrics(
        self.y_test, self.predictions['XGBoost'], 'XGBoost'
    )
    print(f"XGBoost MAE: {self.metrics['XGBoost']['MAE']:.2f}")

MLForecastingPipeline.train_xgboost = train_xgboost


In [13]:

"""
Cell 11: Add LightGBM training method
"""
def train_lightgbm(self, tune_hyperparams=True):
    """Train LightGBM with optional hyperparameter tuning"""
    print("\n" + "="*50)
    print("Training LightGBM...")

    if tune_hyperparams:
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            }

            model = lgb.LGBMRegressor(**params, random_state=42, verbose=-1)

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []
            for train_idx, val_idx in tscv.split(self.X_train):
                model.fit(self.X_train.iloc[train_idx], self.y_train.iloc[train_idx])
                pred = model.predict(self.X_train.iloc[val_idx])
                scores.append(mean_absolute_error(self.y_train.iloc[val_idx], pred))

            return np.mean(scores)

        study = optuna.create_study(direction='minimize')
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study.optimize(objective, n_trials=10)

        self.best_params['LightGBM'] = study.best_params
        self.models['LightGBM'] = lgb.LGBMRegressor(**study.best_params, random_state=42, verbose=-1)
        self.models['LightGBM'].fit(self.X_train, self.y_train)
        print(f"Best params: {study.best_params}")
    else:
        self.models['LightGBM'] = lgb.LGBMRegressor(
            n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42, verbose=-1
        )
        self.models['LightGBM'].fit(self.X_train, self.y_train)

    self.predictions['LightGBM'] = self.models['LightGBM'].predict(self.X_test)
    self.metrics['LightGBM'] = self.calculate_metrics(
        self.y_test, self.predictions['LightGBM'], 'LightGBM'
    )
    print(f"LightGBM MAE: {self.metrics['LightGBM']['MAE']:.2f}")

MLForecastingPipeline.train_lightgbm = train_lightgbm


In [14]:

"""
Cell 12: Add CatBoost training method
"""
def train_catboost(self, tune_hyperparams=True):
    """Train CatBoost with optional hyperparameter tuning"""
    print("\n" + "="*50)
    print("Training CatBoost...")

    cat_features = ['dayofweek', 'month', 'quarter', 'is_weekend', 'is_holiday']
    cat_indices = [self.X_train.columns.get_loc(col) for col in cat_features
                   if col in self.X_train.columns]

    if tune_hyperparams:
        def objective(trial):
            params = {
                'iterations': trial.suggest_int('iterations', 100, 300),
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            }

            model = CatBoostRegressor(**params, random_state=42, verbose=False)

            tscv = TimeSeriesSplit(n_splits=3)
            scores = []
            for train_idx, val_idx in tscv.split(self.X_train):
                model.fit(self.X_train.iloc[train_idx], self.y_train.iloc[train_idx], cat_features=cat_indices)
                pred = model.predict(self.X_train.iloc[val_idx])
                scores.append(mean_absolute_error(self.y_train.iloc[val_idx], pred))

            return np.mean(scores)

        study = optuna.create_study(direction='minimize')
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study.optimize(objective, n_trials=10)

        self.best_params['CatBoost'] = study.best_params
        self.models['CatBoost'] = CatBoostRegressor(**study.best_params, random_state=42, verbose=False)
        self.models['CatBoost'].fit(self.X_train, self.y_train, cat_features=cat_indices)
        print(f"Best params: {study.best_params}")
    else:
        self.models['CatBoost'] = CatBoostRegressor(
            iterations=200, depth=6, learning_rate=0.1, random_state=42, verbose=False
        )
        self.models['CatBoost'].fit(self.X_train, self.y_train, cat_features=cat_indices)

    self.predictions['CatBoost'] = self.models['CatBoost'].predict(self.X_test)
    self.metrics['CatBoost'] = self.calculate_metrics(
        self.y_test, self.predictions['CatBoost'], 'CatBoost'
    )
    print(f"CatBoost MAE: {self.metrics['CatBoost']['MAE']:.2f}")

MLForecastingPipeline.train_catboost = train_catboost


In [15]:

"""
Cell 13: Add LSTM training method with sequence optimization
"""
def prepare_sequences(self, X, y, sequence_length=30):
    """Prepare sequences for LSTM/GRU models"""
    X_scaled = self.scaler_X.fit_transform(X)
    y_scaled = self.scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

    X_seq, y_seq = [], []
    for i in range(sequence_length, len(X_scaled)):
        X_seq.append(X_scaled[i-sequence_length:i])
        y_seq.append(y_scaled[i])

    return np.array(X_seq), np.array(y_seq)

def train_lstm(self, tune_hyperparams=False):
    """Train LSTM model"""
    print("\n" + "="*50)
    print("Training LSTM...")
    print("🚀 Using GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "Using CPU")

    sequence_length = 30  # Fixed for simplicity

    # Prepare sequences
    X_train_seq, y_train_seq = self.prepare_sequences(self.X_train, self.y_train, sequence_length)

    # Prepare test sequences
    X_combined = pd.concat([self.X_train, self.X_test])
    y_combined = pd.concat([self.y_train, self.y_test])

    X_combined_scaled = self.scaler_X.fit_transform(X_combined)

    test_start_idx = len(self.X_train)

    X_test_seq = []
    for i in range(test_start_idx, len(X_combined_scaled)):
        if i >= sequence_length:
            X_test_seq.append(X_combined_scaled[i-sequence_length:i])

    X_test_seq = np.array(X_test_seq)

    # Build model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    early_stop = EarlyStopping(patience=10, restore_best_weights=True, verbose=0)

    model.fit(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

    self.models['LSTM'] = model

    # Predictions
    lstm_pred_scaled = model.predict(X_test_seq, verbose=0)
    lstm_pred = self.scaler_y.inverse_transform(lstm_pred_scaled).flatten()

    y_test_aligned = self.y_test.iloc[-len(lstm_pred):].values

    self.predictions['LSTM'] = lstm_pred
    self.metrics['LSTM'] = self.calculate_metrics(y_test_aligned, lstm_pred, 'LSTM')
    print(f"LSTM MAE: {self.metrics['LSTM']['MAE']:.2f}")

    self.y_test_lstm = y_test_aligned

MLForecastingPipeline.prepare_sequences = prepare_sequences
MLForecastingPipeline.train_lstm = train_lstm


In [16]:

"""
Cell 14: Add GRU training method
"""
def train_gru(self, tune_hyperparams=False):
    """Train GRU model"""
    print("\n" + "="*50)
    print("Training GRU...")
    print("🚀 Using GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "Using CPU")

    sequence_length = 30

    # Prepare sequences
    X_train_seq, y_train_seq = self.prepare_sequences(self.X_train, self.y_train, sequence_length)

    # Prepare test sequences
    X_combined = pd.concat([self.X_train, self.X_test])
    X_combined_scaled = self.scaler_X.fit_transform(X_combined)

    test_start_idx = len(self.X_train)

    X_test_seq = []
    for i in range(test_start_idx, len(X_combined_scaled)):
        if i >= sequence_length:
            X_test_seq.append(X_combined_scaled[i-sequence_length:i])

    X_test_seq = np.array(X_test_seq)

    # Build model
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        Dropout(0.2),
        GRU(32),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    early_stop = EarlyStopping(patience=10, restore_best_weights=True, verbose=0)

    model.fit(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

    self.models['GRU'] = model

    # Predictions
    gru_pred_scaled = model.predict(X_test_seq, verbose=0)
    gru_pred = self.scaler_y.inverse_transform(gru_pred_scaled).flatten()

    y_test_aligned = self.y_test.iloc[-len(gru_pred):].values

    self.predictions['GRU'] = gru_pred
    self.metrics['GRU'] = self.calculate_metrics(y_test_aligned, gru_pred, 'GRU')
    print(f"GRU MAE: {self.metrics['GRU']['MAE']:.2f}")

    self.y_test_gru = y_test_aligned

MLForecastingPipeline.train_gru = train_gru


In [17]:

"""
Cell 15: Add Prophet training method - with all fixes applied
"""
def train_prophet(self, tune_hyperparams=False):
    """Train Prophet model - FIXED VERSION"""
    print("\n" + "="*50)
    print("Training Prophet...")

    # Prepare data for Prophet - FIXED: using self.train_dates
    train_prophet = pd.DataFrame({
        'ds': self.train_dates,
        'y': self.y_train.values
    })

    # Suppress Prophet output
    import logging
    logging.getLogger('prophet').setLevel(logging.WARNING)
    logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

    self.models['Prophet'] = Prophet(
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=1.0,
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True
    )

    # Fit model - FIXED: no verbose parameter
    self.models['Prophet'].fit(train_prophet)

    # Make predictions - FIXED: using self.test_dates
    future = pd.DataFrame({'ds': self.test_dates})
    forecast = self.models['Prophet'].predict(future)

    self.predictions['Prophet'] = forecast['yhat'].values
    self.metrics['Prophet'] = self.calculate_metrics(
        self.y_test, self.predictions['Prophet'], 'Prophet'
    )
    print(f"Prophet MAE: {self.metrics['Prophet']['MAE']:.2f}")

MLForecastingPipeline.train_prophet = train_prophet


In [18]:

"""
Cell 16: Initialize the ML pipeline with dates
"""
pipeline = MLForecastingPipeline(
    X_train, X_test, y_train, y_test,
    train_dates, test_dates  # Fixed: passing dates
)
print("✅ Pipeline initialized successfully!")


✅ Pipeline initialized successfully!


In [19]:
pipeline.train_random_forest(tune_hyperparams=True)


Training Random Forest...
Best params: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10}
RandomForest MAE: 193.39


In [20]:
pipeline.train_xgboost(tune_hyperparams=True)

[I 2025-09-19 21:28:59,305] A new study created in memory with name: no-name-ecfda0b0-ff95-4870-aea8-df6a566e941f



Training XGBoost...
Best params: {'n_estimators': 249, 'max_depth': 6, 'learning_rate': 0.1098640499537224, 'subsample': 0.6816336126201117, 'colsample_bytree': 0.8922509087161615}
XGBoost MAE: 184.71


In [21]:
pipeline.train_lightgbm(tune_hyperparams=True)


Training LightGBM...
Best params: {'n_estimators': 255, 'max_depth': 3, 'learning_rate': 0.06708713270745811, 'num_leaves': 84}
LightGBM MAE: 170.80


In [22]:
pipeline.train_catboost(tune_hyperparams=True)


Training CatBoost...
Best params: {'iterations': 170, 'depth': 5, 'learning_rate': 0.04173487655191767}
CatBoost MAE: 291.12


In [23]:

"""
Cell 21: Save checkpoint after tree models
"""
checkpoint = {
    'models': pipeline.models.copy(),
    'predictions': pipeline.predictions.copy(),
    'metrics': pipeline.metrics.copy(),
    'best_params': pipeline.best_params.copy()
}

with open('tree_models_checkpoint.pkl', 'wb') as f:
    pickle.dump(checkpoint, f)
print("✅ Tree models checkpoint saved!")


✅ Tree models checkpoint saved!


In [24]:
pipeline.train_lstm(tune_hyperparams=False)


Training LSTM...
🚀 Using GPU
Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0775 - mae: 0.2088 - val_loss: 0.0487 - val_mae: 0.1756
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0158 - mae: 0.0972 - val_loss: 0.0533 - val_mae: 0.1841
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0131 - mae: 0.0874 - val_loss: 0.0480 - val_mae: 0.1716
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0130 - mae: 0.0876 - val_loss: 0.0509 - val_mae: 0.1768
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0103 - mae: 0.0794 - val_loss: 0.0431 - val_mae: 0.1632
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0099 - mae: 0.0768 - val_loss: 0.0438 - val_mae: 0.1640
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0087 - mae: 0.0723 - val_loss: 0.0453 - val_mae: 0.1654
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0085 - mae: 0.0689 - val_loss: 0.0442 - val_mae: 0.1643
Epoch 9/50
17/17 ━━━━━━━━━━━━━━━━━

In [25]:
pipeline.train_gru(tune_hyperparams=False)


Training GRU...
🚀 Using GPU
Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0913 - mae: 0.2369 - val_loss: 0.1132 - val_mae: 0.2995
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0289 - mae: 0.1365 - val_loss: 0.0899 - val_mae: 0.2640
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0243 - mae: 0.1198 - val_loss: 0.0535 - val_mae: 0.1967
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0199 - mae: 0.1102 - val_loss: 0.0492 - val_mae: 0.1903
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0120 - mae: 0.0873 - val_loss: 0.0391 - val_mae: 0.1678
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0134 - mae: 0.0890 - val_loss: 0.0326 - val_mae: 0.1534
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0129 - mae: 0.0861 - val_loss: 0.0286 - val_mae: 0.1404
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0107 - mae: 0.0800 - val_loss: 0.0310 - val_mae: 0.1485
Epoch 9/50
17/17 ━━━━━━━━━━━━━━━━━━

In [26]:
pipeline.train_prophet(tune_hyperparams=False)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpz01qihwv/3sz2bg00.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpz01qihwv/c7l5ns3o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92295', 'data', 'file=/tmp/tmpz01qihwv/3sz2bg00.json', 'init=/tmp/tmpz01qihwv/c7l5ns3o.json', 'output', 'file=/tmp/tmpz01qihwv/prophet_modelhzyyor3d/prophet_model-20250919213406.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:34:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:34:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



Training Prophet...
Prophet MAE: 2471.75


In [27]:

"""
Cell 25: Create metrics summary DataFrame
"""
metrics_df = pd.DataFrame(pipeline.metrics).T
metrics_df = metrics_df.sort_values('MAE')

print("\n" + "="*50)
print("MODEL PERFORMANCE COMPARISON")
print("="*50)
display(metrics_df.round(2))



MODEL PERFORMANCE COMPARISON


,Model,MAE,RMSE,MAPE,R2
LightGBM,LightGBM,170.804606,223.139681,2.313382,0.986293
XGBoost,XGBoost,184.71196,259.562026,2.506934,0.981453
RandomForest,RandomForest,193.39313,251.502817,2.730669,0.982587
CatBoost,CatBoost,291.123734,373.741418,3.996613,0.961548
GRU,GRU,1061.042847,1307.138191,14.678432,0.529648
LSTM,LSTM,2173.521484,2917.93026,25.854269,-1.34385
Prophet,Prophet,2471.75477,2843.530723,31.954086,-1.225849


In [28]:
"""
Cell 26: Create ensemble models - DEBUGGED VERSION
"""
print("\n" + "="*50)
print("CREATING ENSEMBLE MODELS")
print("="*50)

# First, let's debug what we're working with
print("Debugging metrics_df structure:")
print(f"metrics_df shape: {metrics_df.shape}")
print(f"metrics_df columns: {metrics_df.columns.tolist()}")
print(f"metrics_df index: {metrics_df.index.tolist()}")

# Simple averaging ensemble (top 3 models)
top_3_models = metrics_df.head(3).index.tolist()
print(f"\nTop 3 models by MAE: {top_3_models}")

ensemble_predictions = {}

# 1. Simple Average of Top 3
if len(top_3_models) >= 3:
    aligned_preds = []
    min_length = min([len(pipeline.predictions[m]) for m in top_3_models])
    print(f"Min prediction length: {min_length}")

    for model in top_3_models:
        pred_len = len(pipeline.predictions[model])
        print(f"{model} prediction length: {pred_len}")
        aligned_preds.append(pipeline.predictions[model][-min_length:])

    ensemble_predictions['Ensemble_Avg_Top3'] = np.mean(aligned_preds, axis=0)

    y_test_ensemble = y_test.iloc[-min_length:].values
    mae_ensemble = mean_absolute_error(y_test_ensemble, ensemble_predictions['Ensemble_Avg_Top3'])
    print(f"\nEnsemble (Average Top 3) MAE: {mae_ensemble:.2f}")

    # 2. Weighted Average - COMPLETELY FIXED
    # Extract MAE values more carefully
    mae_list = []
    for model in top_3_models:
        mae_val = metrics_df.loc[model, 'MAE']
        print(f"{model} MAE: {mae_val}, type: {type(mae_val)}")
        mae_list.append(float(mae_val))  # Ensure it's a float

    mae_array = np.array(mae_list)
    print(f"MAE array: {mae_array}")

    # Calculate weights
    weights = 1.0 / mae_array
    weights = weights / weights.sum()
    print(f"Raw weights: {weights}")

    # Display weights without using np.round
    weights_display = {model: float(f"{w:.3f}") for model, w in zip(top_3_models, weights)}
    print(f"Weights for top 3 models: {weights_display}")

    # Calculate weighted ensemble
    weighted_ensemble = np.average(np.array(aligned_preds), axis=0, weights=weights)
    ensemble_predictions['Ensemble_Weighted'] = weighted_ensemble

    mae_weighted = mean_absolute_error(y_test_ensemble, weighted_ensemble)
    print(f"Ensemble (Weighted) MAE: {mae_weighted:.2f}")
else:
    print(f"⚠️ Only {len(top_3_models)} models available, need at least 3 for ensemble")



CREATING ENSEMBLE MODELS
Debugging metrics_df structure:
metrics_df shape: (7, 5)
metrics_df columns: ['Model', 'MAE', 'RMSE', 'MAPE', 'R2']
metrics_df index: ['LightGBM', 'XGBoost', 'RandomForest', 'CatBoost', 'GRU', 'LSTM', 'Prophet']

Top 3 models by MAE: ['LightGBM', 'XGBoost', 'RandomForest']
Min prediction length: 180
LightGBM prediction length: 180
XGBoost prediction length: 180
RandomForest prediction length: 180

Ensemble (Average Top 3) MAE: 154.40
LightGBM MAE: 170.8046055777117, type: <class 'float'>
XGBoost MAE: 184.7119598388672, type: <class 'float'>
RandomForest MAE: 193.39313039179444, type: <class 'float'>
MAE array: [170.80460558 184.71195984 193.39313039]
Raw weights: [0.35613717 0.32932285 0.31453997]
Weights for top 3 models: {'LightGBM': 0.356, 'XGBoost': 0.329, 'RandomForest': 0.315}
Ensemble (Weighted) MAE: 153.52


In [29]:

"""
Cell 27: Create final results summary
"""
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

# Combine all metrics
all_metrics = pipeline.metrics.copy()

# Add ensemble metrics if they exist
if 'Ensemble_Avg_Top3' in ensemble_predictions:
    all_metrics['Ensemble_Avg_Top3'] = {
        'Model': 'Ensemble_Avg_Top3',
        'MAE': mae_ensemble,
        'RMSE': np.sqrt(mean_squared_error(y_test_ensemble, ensemble_predictions['Ensemble_Avg_Top3'])),
        'MAPE': mean_absolute_percentage_error(y_test_ensemble, ensemble_predictions['Ensemble_Avg_Top3']) * 100,
        'R2': 1 - (np.sum((y_test_ensemble - ensemble_predictions['Ensemble_Avg_Top3']) ** 2) /
                   np.sum((y_test_ensemble - y_test_ensemble.mean()) ** 2))
    }

if 'Ensemble_Weighted' in ensemble_predictions:
    all_metrics['Ensemble_Weighted'] = {
        'Model': 'Ensemble_Weighted',
        'MAE': mae_weighted,
        'RMSE': np.sqrt(mean_squared_error(y_test_ensemble, ensemble_predictions['Ensemble_Weighted'])),
        'MAPE': mean_absolute_percentage_error(y_test_ensemble, ensemble_predictions['Ensemble_Weighted']) * 100,
        'R2': 1 - (np.sum((y_test_ensemble - ensemble_predictions['Ensemble_Weighted']) ** 2) /
                   np.sum((y_test_ensemble - y_test_ensemble.mean()) ** 2))
    }

# Final summary table
final_results = pd.DataFrame(all_metrics).T
final_results = final_results.sort_values('MAE')

print("\nFinal Performance Ranking (by MAE):")
display(final_results[['Model', 'MAE', 'RMSE', 'MAPE', 'R2']].round(2))

# Best model
best_model = final_results.iloc[0]['Model']
best_mae = final_results.iloc[0]['MAE']
print(f"\n🏆 BEST MODEL: {best_model}")
print(f"   MAE: {best_mae:.2f}")

# Comparison with SARIMA baseline
sarima_mae = 156
improvement = ((sarima_mae - best_mae) / sarima_mae) * 100
print(f"\n📊 Improvement over SARIMA (MAE=156): {improvement:.1f}%")



FINAL RESULTS SUMMARY

Final Performance Ranking (by MAE):


,Model,MAE,RMSE,MAPE,R2
Ensemble_Weighted,Ensemble_Weighted,153.522629,203.487639,2.116483,0.988601
Ensemble_Avg_Top3,Ensemble_Avg_Top3,154.399304,204.775165,2.129251,0.988457
LightGBM,LightGBM,170.804606,223.139681,2.313382,0.986293
XGBoost,XGBoost,184.71196,259.562026,2.506934,0.981453
RandomForest,RandomForest,193.39313,251.502817,2.730669,0.982587
CatBoost,CatBoost,291.123734,373.741418,3.996613,0.961548
GRU,GRU,1061.042847,1307.138191,14.678432,0.529648
LSTM,LSTM,2173.521484,2917.93026,25.854269,-1.34385
Prophet,Prophet,2471.75477,2843.530723,31.954086,-1.225849



🏆 BEST MODEL: Ensemble_Weighted
   MAE: 153.52

📊 Improvement over SARIMA (MAE=156): 1.6%


In [30]:

"""
Cell 28: Save all models and results
"""
# Save tree-based models and Prophet
models_to_save = {}
for model_name, model in pipeline.models.items():
    if model_name not in ['LSTM', 'GRU']:
        models_to_save[model_name] = model

with open('ml_models_final.pkl', 'wb') as f:
    pickle.dump(models_to_save, f)
print("✅ Tree models saved to ml_models_final.pkl")

# Save neural network models
if 'LSTM' in pipeline.models:
    pipeline.models['LSTM'].save('lstm_model_final.h5')
    print("✅ LSTM saved to lstm_model_final.h5")

if 'GRU' in pipeline.models:
    pipeline.models['GRU'].save('gru_model_final.h5')
    print("✅ GRU saved to gru_model_final.h5")

# Save results
results = {
    'metrics': all_metrics,
    'predictions': pipeline.predictions,
    'best_params': pipeline.best_params,
    'final_results': final_results.to_dict()
}

with open('ml_results_final.pkl', 'wb') as f:
    pickle.dump(results, f)
print("✅ Results saved to ml_results_final.pkl")

print("\n" + "="*60)
print("ML FORECASTING PIPELINE COMPLETE!")
print("="*60)


✅ Tree models saved to ml_models_final.pkl
✅ LSTM saved to lstm_model_final.h5
✅ GRU saved to gru_model_final.h5
✅ Results saved to ml_results_final.pkl

ML FORECASTING PIPELINE COMPLETE!


In [31]:

"""
RECOVERY CELL - Run this after restart to restore progress
"""
# # Uncomment below to recover from checkpoint
# import pickle
#
# # Load checkpoint
# with open('tree_models_checkpoint.pkl', 'rb') as f:
#     checkpoint = pickle.load(f)
#
# # Reinitialize pipeline
# pipeline = MLForecastingPipeline(X_train, X_test, y_train, y_test, train_dates, test_dates)
# pipeline.models = checkpoint['models']
# pipeline.predictions = checkpoint['predictions']
# pipeline.metrics = checkpoint['metrics']
# pipeline.best_params = checkpoint['best_params']
#
# print(f"✅ Restored {len(pipeline.models)} models from checkpoint")
# print(f"Models loaded: {list(pipeline.models.keys())}")


'\nRECOVERY CELL - Run this after restart to restore progress\n'